In [14]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import matplotlib.pyplot as plt

In [15]:
FN = "regression"

In [16]:
with open("data.json","r") as data:
    data = json.load(data)
# Shuffle the training set
inputs,outputs= np.array(data['inputs']),np.array(data['outputs'])
aa = int(len(inputs)*.9)
train_inputs,train_outputs=inputs[:aa],outputs[:aa] 
test_inputs,test_outputs=inputs[aa:],outputs[aa:] 
order = np.argsort(np.random.random(train_outputs.shape))
train_inputs = train_inputs[order]
train_outputs = train_outputs[order]

In [17]:
mean = train_inputs.mean(axis=0)
std = train_inputs.std(axis=0)
train_inputs = (train_inputs - mean) / std
test_inputs = (test_inputs - mean) / std

In [18]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(train_inputs.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae','acc'])
  return model

model = build_model()

In [19]:
EPOCHS = 500

In [20]:
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
history = model.fit(train_inputs, train_outputs, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[early_stop])

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                896       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [22]:
model_trained = model.to_json()

In [23]:
with open('regression_model.json',"w") as file:
    json.dump(model_trained,file)

In [24]:
model.save_weights("regression_weights",save_format="h5")

In [25]:
with open('regression_history.json',"w") as file:
    json.dump(history.history,file)

In [26]:
len(history.history['loss'])

114